In [4]:
import os
import csv

In [14]:
import os
import csv

def read_first_row(file_path):
    """
    Read the first row from the columns "DetectorID" and "Location" in a CSV file.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        tuple: A tuple containing the "DetectorID" and "Location" from the first row.
    """
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader)
        detector_id = first_row.get('DetectorID', '')
        location = first_row.get('Location', '')
    return detector_id, location

def get_unique_data(directory1, directory2):
    unique_data = set()
    
    for directory in [directory1, directory2]:
        for filename in os.listdir(directory):
            if filename.endswith('.csv'):
                file_path = os.path.join(directory, filename)
                detector_id, location = read_first_row(file_path)
                if detector_id and location:  # Ensure both values are non-empty
                    unique_data.add((detector_id, location))
    
    return unique_data

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['DetectorID', 'Location'])
        for detector_id, location in data:
            writer.writerow([detector_id, location])

def main():
    directory1 = 'E:/xie/Sensor Files/1. raw data - separated sensor files - num checked out with the db/2018'
    directory2 = 'E:/xie/Sensor Files/1. raw data - separated sensor files - num checked out with the db/2019'
    output_file = 'E:/xie/sensor_files_xie.csv'

    unique_data = get_unique_data(directory1, directory2)

    write_to_csv(unique_data, output_file)

    print("Unique data has been written to", output_file)

if __name__ == "__main__":
    main()


Unique data has been written to E:/xie/sensor_files_xie.csv


In [16]:
import csv

def read_csv(file_path):
    data = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row['DetectorID'])
    return data

def compare_sensor_names(csv_file1, csv_file2):
    sensor_names_file1 = read_csv(csv_file1)
    sensor_names_file2 = read_csv(csv_file2)

    # Replace underscores with dots in sensor names from file 2
    sensor_names_file2 = [name.replace('_', '.') for name in sensor_names_file2]

    common_sensor_names = set(sensor_names_file1) & set(sensor_names_file2)
    return common_sensor_names

def main():
    csv_file_created = 'E:/xie/sensor_files_xie.csv'
    another_csv_file = 'E:/xie/Chloe - Reduced Detector Data - all_combined.csv'  # Provide the path to another CSV file

    common_sensor_names = compare_sensor_names(csv_file_created, another_csv_file)
    
    print(f"Number of sensor names found in both files: {len(common_sensor_names)}")
    print("Common sensor names:")
    for sensor_name in common_sensor_names:
        print(sensor_name)

if __name__ == "__main__":
    main()


Number of sensor names found in both files: 790
Common sensor names:
74.1.22
9.1.3
407.2.233
222.1.172
443.1.344
612.1.467
523.2.84
171.1.196
182.1.75
496.2.266
126.1.49
156.2.57
650.1.492
363.2.161
480.3.123
491.1.237
198.1.70
355.3.155
454.2.513
665.2.462
357.1.312
514.2.158
391.1.415
394.1.434
457.3.52
506.2.271
215.2.362
448.2.354
212.2.364
470.2.244
516.2.149
212.4.363
490.1.254
648.1.488
132.1.80
374.3.287
641.1.470
423.1.329
201.1.39
419.1.234
609.1.479
612.2.465
113.2.45
159.2.57
606.2.489
383.2.375
363.1.297
519.2.144
529.1.81
677.1.449
377.1.162
499.2.542
378.4.130
520.2.155
694.1.538
656.1.503
417.2.230
326.2.195
442.2.340
362.3.129
50.2.12
5.1.6
203.2.67
643.1.474
476.1.261
89.1.28
203.3.68
596.1.485
444.2.100
115.1.45
555.1.405
437.2.328
505.2.273
375.2.164
15.1.6
429.1.345
147.2.56
49.3.15
387.1.407
137.1.80
329.2.198
453.1.509
421.1.505
80.2.24
135.1.80
203.4.69
58.2.17
642.1.476
368.3.381
396.3.246
631.1.496
180.2.75
417.1.216
563.1.133
10.2.268
210.2.59
364.1.300
503.2